In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pickle
import seaborn as sns
import numpy as np
import csv
import random

In [2]:
# If CUDA supported GPU available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Function to load the custom test dataset
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
# Load the custom test dataset
test_data = unpickle('cifar_test_nolabels.pkl')

In [5]:
from torchvision import transforms

# Setting up the transformation for test data.
# It's crucial to apply the same normalization as used during training.
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


In [6]:
test_images = test_data[b'data'].reshape((-1, 3, 32, 32)).astype(np.float32) / 255.0
test_images = torch.tensor(test_images)

# Define mean and std
mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(1, 3, 1, 1)
std = torch.tensor([0.2023, 0.1994, 0.2010]).view(1, 3, 1, 1)

# Normalize the test images
preprocessed_images = (test_images - mean) / std

# Create a TensorDataset and DataLoader for the normalized test images
custom_testset = torch.utils.data.TensorDataset(preprocessed_images)
custom_testloader = DataLoader(custom_testset, batch_size=100, shuffle=False)


In [7]:
# Creating a TensorDataset and DataLoader for the preprocessed test data
custom_testset = torch.utils.data.TensorDataset(preprocessed_images)
custom_testloader = DataLoader(custom_testset, batch_size=100, shuffle=False)

In [8]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out


In [9]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64  # Adjust this to match parameter budget

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # Adjust block numbers to fit parameter budget
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)

        # Removed layer4 to save parameters and fit the budget

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Ensure the linear layer matches the output of layer3
        self.linear = nn.Linear(256 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)

        # Removed the pass through the removed layer4

        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNetCustom():
    return ResNet(BasicBlock, [2, 2, 3])  # Adjusted block numbers for the budget

model = ResNetCustom().to(device)


In [10]:
model = ResNetCustom().to(device)

# Define the path to your model weights
model_weights_path = 'model_weights.pth'  # Replace this with the actual path

# Load the trained model weights
model.load_state_dict(torch.load(model_weights_path, map_location=device))

# Set the model to evaluation mode
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [11]:
# Set the model to evaluation mode
model.eval()

predictions = []
with torch.no_grad():
    for data in custom_testloader:
        images = data[0].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().tolist())

In [12]:
image_ids = np.arange(len(predictions))

output_csv_path = 'predictions.csv'

with open(output_csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID', 'Labels'])  # Corrected column names
    for img_id, prediction in zip(image_ids, predictions):
        writer.writerow([img_id, prediction])

print(f'Predictions have been saved to {output_csv_path}')

Predictions have been saved to predictions.csv
